Para o projeto, vamos utilizar a biblioteca PySUS criada por fccoelho.

Mais informações podem ser encontradas através dos links abaixo:

Github:
https://github.com/AlertaDengue/PySUS

Documentação:
https://pysus.readthedocs.io/en/latest/tutorials.html#working-with-sih-data

Para realizar o download, precisaremos das seguintes bibliotecas e métodos:

In [3]:
from pysus.online_data.SIH import download
from pysus.online_data import parquets_to_dataframe
import pandas as pd
import csv

Como o dataset é muito grande e só utilizaremos algumas colunas no projeto, vamos primeiro listar e escolher as colunas que serão utilizadas.

In [10]:
# Faz download de um pequeno subset dos dados (escolhi arbitrariamente jan/2015).
df_test = parquets_to_dataframe(download(states='AL', years=2015, months=1))

# Cria uma lista de listas com os nomes das colunas, visto que usaremos a função writerows.
lista = [[coluna] for coluna in df_test.columns.tolist()]

# Cria um csv com os nomes das colunas.
with open('nome_colunas.csv', 'w', newline='') as arquivo:
    csv_writer = csv.writer(arquivo)
    csv_writer.writerows(lista)

O significado de cada coluna pode ser encontrado no link abaixo.
[ INSERIR LINK ]

Após analizar as colunas, foram escolhidas as seguintes colunas:

In [1]:
colunas = [
    'UF_ZI',
    'ANO_CMPT',
    'MES_CMPT',
    'ESPEC',
    'CGC_HOSP',
    'N_AIH',
    'IDENT',
    'CEP',
    'MUNIC_RES'
]

Como para o projeto só utilizaremos dados de Maceió, podemos filtrar as linhas através do valor da coluna 'MUNIC_RES' que é o código do municipio no IBGE.

Através do site abaixo é possível consultar o código dos municípios no IBGE.  
https://www.ibge.gov.br/explica/codigos-dos-municipios.php

O código de Maceió é 2704302

In [4]:
df_consolidado = pd.DataFrame()

for ano in range(2014, 2023+1): # +1 visto que o método range() é não inclusivo
    for mes in range(1,12+1):
        print(f'Iniciando download: {mes}/{ano}...', end=' ')
        
        dados = download(states='AL', years=ano, months=mes)
        print(f'Download realizado com sucesso.')

        # Converte os dados baixados em um dataframe
        df = parquets_to_dataframe(dados)
        
        # Filtra apenas registros de Maceió
        df = df[df['MUNIC_RES']==2704302]

        # Filtra apenas as colunas 
        df = df[colunas] 
        
        df_consolidado = pd.concat([df_consolidado, df])
        
        print(f'Dataframe agora possuí {df.shape[0]} linhas.\n')
        
print('Download concluído')

Iniciando download: 1/2014... Download realizado com sucesso.
Dataframe agora possuí 13964 linhas.

Iniciando download: 2/2014... Download realizado com sucesso.
Dataframe agora possuí 26897 linhas.

Iniciando download: 1/2015... Download realizado com sucesso.
Dataframe agora possuí 40197 linhas.

Iniciando download: 2/2015... Download realizado com sucesso.
Dataframe agora possuí 53706 linhas.



,UF_ZI,ANO_CMPT,MES_CMPT,ESPEC,CGC_HOSP,N_AIH,IDENT,CEP,MUNIC_RES
0,270000,2014,01,01,12200259000408,2714103222512,1,57303785,270030
1,270000,2014,01,03,12200259000408,2714103221742,1,57260000,270420
2,270000,2014,01,03,12200259000408,2714103221753,1,57313060,270030
3,270000,2014,01,03,12200259000408,2714103221764,1,57560000,270250
4,270000,2014,01,03,12200259000408,2714103221775,1,57300620,270030


In [5]:
df.to_csv('SIH_SUS_AL-2014-2023.csv', sep=',', encoding='utf-8', index=False, compression='gzip')

Caso você tenha algum problema em realizar o download dessa forma, também é possível fazer o download de todos os arquivos dbc e converte-los.

In [ ]:
import pysus.utilities.readdbc as dbc
df = dbc.read_dbc('dbc_files/RDAL2201.dbc')
df.head()